## BETTER WAY 25 - `super`로 부모 클래스를 초기화하자
## Item25 - Initialize Parent Classes with `super`

예전에는 자식 클래스에서 부모 클래스의 `__init__` 메소드를 직접 호출하는 방법으로 부모 클래스를 초기화했다.

In [1]:
class MyBaseClass(object):
    def __init__(self, value):
        self.value = value
        
        
class MyChildClass(MyBaseClass):
    def __init__(self):
        # 부모클래스의 __init()__ 메소드 직접호출
        MyBaseClass.__init__(self, 5) 

이 방법은 간단한 계층 구조에는 잘 동작하지만 많은 경우 제대로 동작하지 못한다.

클래스가 다중상속의 영향을 받는다면 슈퍼클래스(부모 클래스)의 `__init__` 메소드를 직접 호출하는 방법은 예기치 못한 동작을 발생시킬 수 있다.

한 가지 문제는 `__init__`의 호출 순서가 모든 서브클래스에 걸쳐 명시되어 있지 않다는 것이다. 아래의 예제 코드는 인스턴스의 value 필드로 연산을 수행하는 부모 클래스 두 개를 나타낸 것이다.

In [2]:
class TimesTwo(object):
    def __init__(self):
        self.value *= 2
        
    
class PlusFive(object):
    def __init__(self):
        self.value += 5

아래의 예제 클래스는 한 가지 순서로 부모 클래스들을 정의한다.

In [3]:
class OneWay(MyBaseClass, TimesTwo, PlusFive):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

이 클래스의 인스턴스를 생성하면 아래와 같이 출력결과가 부모 클래스의 순서와 일치하는 결과가 만들어진다.

In [4]:
foo = OneWay(5)
print('First oderting is (5 * 2) + 5=', foo.value)

First oderting is (5 * 2) + 5= 15


다음은 같은 부모 클래스들을 다른 순서로 정의한 클래스다.

In [5]:
class AnotherWay(MyBaseClass, PlusFive, TimesTwo):
    # 위의 OneWay class 와 다른 순서인
    # MyBaseClass -> PlusFive -> TimesTwo로 정의함
    def __init__(self, value):
        # OneWay class와 같은 순서로 초기화
        MyBaseClass.__init__(self, value)
        TimesTwo.__init__(self)
        PlusFive.__init__(self)

하지만 부모 클래스 생성자 PlusFile.`__init__`, TimesTwo.`__init__`를 이전과 같은 순서로 호출한다. 이 클래스의 동작은 부모 클래스를 정의한 순서와 일치하지 않는다.

In [6]:
bar = AnotherWay(5)
print('First oderting is (5 + 5) * 2 !=', bar.value)

First oderting is (5 + 5) * 2 != 15


다른 문제는 다이아몬드 상속(diamond inheritance)이다. 다이아몬드 상속은 서브클래스가 계층 구조에서 같은 슈퍼클래스를 둔 서로 다른 두 클래스에서 상속받을 때 발생한다. 다이아몬드 상속은 공통 슈퍼클래스의 `__init__` 메서드를 여러 번 실행하게 해서 예상치 못한 동작을 일으킨다.

예를 들어 MyBaseClass에서 상속받는 자식 클래스 두 개를 정의해보자.

In [7]:
class TimesFive(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value *= 5
        
        
class PlusTwo(MyBaseClass):
    def __init__(self, value):
        MyBaseClass.__init__(self, value)
        self.value += 2

다음으로 이 두 클래스 모두에서 상속받는 자식 클래스를 정의하여 MyBaseClass를 다이아몬드의 꼭대기로 만든다.

In [8]:
class ThisWay(TimesFive, PlusTwo):
    def __init__(self, value):
        TimesFive.__init__(self, value)
        PlusTwo.__init__(self, value)
        
        
foo = ThisWay(5)
print("Should be (5 * 5) + 2 = 27 but is", foo.value)

Should be (5 * 5) + 2 = 27 but is 7


두 번째 부모 클래스의 생성자 `PlusTwo.__init__`를 호출하는 코드가 있어서 `MyBaseClass.__init__`가 두 번째 호출될 때 self.value를 다시 5로 리셋한다.

파이썬에서는 이 문제를 해결하려고 super라는 내장 함수를 추가하고 메서드 해석 순서(MRO, Method Resolution Order)를 정의했다. MRO는 어떤 슈퍼클래스부터 초기화하는지를 정한다(예를 들면 깊이 우선, 왼쪽에서 오른쪽으로). 또한 다이아몬드 계층 구조에 있는 공통 슈퍼클래스를 단 한 번만 실행하게 한다.

파이썬3에서는 `super`를 인수 없이 호출하면 `__clas__` 와 `self` 를 인수로 넘겨서 호출한 것으로 처리해서 이 문제를 해결한다. 파이썬3에서는 항상 `super` 를 사용해야 한다. `super`는 명확하고 간결하며 항상 제대로 동작한다.

In [9]:
class Explicit(MyBaseClass):
    def __init__(self, value):
        super(__class__, self).__init__(value * 2)
        
    
class Implicit(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 2)
        
print('Explicit(10).value =', Explicit(10).value)
print('Implicit(10).value =', Implicit(10).value)

Explicit(10).value = 20
Implicit(10).value = 20


In [10]:
print('Explicit.mro() :', Explicit.mro())
print('Implicit.mro() :', Implicit.mro())

Explicit.mro() : [<class '__main__.Explicit'>, <class '__main__.MyBaseClass'>, <class 'object'>]
Implicit.mro() : [<class '__main__.Implicit'>, <class '__main__.MyBaseClass'>, <class 'object'>]


아래의 코드는 위에서 보았던 예제인 `TimesFive, PlusTwo` 클래스를 `super()` 를 이용하여 부모클래스를 초기화 해준 코드이다. 

In [11]:
class ReTimesFive(MyBaseClass):
    def __init__(self, value):
        super().__init__(value * 5)
        
        
class RePlusTwo(MyBaseClass):
    def __init__(self, value):
        super().__init__(value + 2)

아래와 같이 `OtherWay` 클래스를 super()를 이용하여 부모클래스를 초기화하면 다이아몬드 상속 문제를 해결할 수 있다.

In [12]:
class OtherWay(ReTimesFive, RePlusTwo):
    def __init__(self, value):
        super().__init__(value)
        
foo = OtherWay(5)
print("Should be (5 * 5) + 2 = 27 so, the answer is", foo.value)

Should be (5 * 5) + 2 = 27 so, the answer is 27


In [13]:
OtherWay.mro()

[__main__.OtherWay,
 __main__.ReTimesFive,
 __main__.RePlusTwo,
 __main__.MyBaseClass,
 object]

### 정리
- 파이썬의 표준 메서드 해석 순서(MRO)는 슈퍼클래스의 초기화 순서와 다이아몬드 상속 문제를 해결한다.
- 항상 내장 함수 super로 부모 클래스를 초기화하자.